In [4]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sqlalchemy
import re
import ast
import math

In [5]:
df = pd.read_csv('esg_dataset_2021_cleansing.csv')
df['noun']  = df['noun'].map(lambda x : ast.literal_eval(x))
df.head(5)

,title,url,press,date,content,noun
0,"올해는 ESG 정보공시 표준화 원년, 압박 더 거세진다",https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,() 산업·금융·법조계 ESG 전문가 대담]/사진제공=이미지투데이2020년은 E...,"[산업, 금융, 법조, ESG, 전문가, 대담, 사진, 제공, 이미지, ESG, 환..."
1,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,() 작년 착한 투자에 30조 몰려인베스코솔라 年 급등美기업 IPO 열기도 뜨거...,"[작년, 투자, 려인, 베스코, 솔라, 年, 급등, 美, 기업, IPO, 로빈, 후..."
2,"[신년사]방문규 수은 행장 ""수출 6000억달러 선봉장될 것""",https://news.naver.com/main/read.naver?mode=LS...,뉴시스,2021.01.01.,() ESG 경영 강화...수출 중소중견기업 맞춤형 지원 방문규 수출입은행장 (사...,"[ESG, 경영, 강화, 수출, 중소, 중견, 기업, 맞춤, 지원, 방문, 규, 수..."
3,[韓산업 대전환 ①] 격변기의 시대…대전환 앞당겼다,https://news.naver.com/main/read.naver?mode=LS...,아이뉴스24,2021.01.01.,() 코로나 팬데믹에 뉴노멀 시대 활짝…대내외 경제 불안 속 미래 먹거리 찾기 총...,"[코로나, 믹, 뉴노, 멀, 시대, 대내외, 경제, 불안, 속, 미래, 먹거리, 총..."
4,"펀드사태 혼쭐난 금융권, 외부 `포청천` 영입",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,"() 4대 금융지주 인사 키워드는우리금융, 감사원 출신 신민철하나銀은 이인영 변호...","[금융, 지주, 인사, 키워드, 금융, 감사원, 출신, 신민철, 銀, 이인영, 변호..."


In [6]:
tokenized_data = []
for noun_list in df['noun']:
    tokenized_data.append(noun_list)
tokenized_data[:3]

[['산업',
  '금융',
  '법조',
  'ESG',
  '전문가',
  '대담',
  '사진',
  '제공',
  '이미지',
  'ESG',
  '환경',
  '사회',
  '가치',
  '지배',
  '구조',
  '논의',
  '원년',
  '평가',
  '포털',
  '사이트',
  '다음',
  'Daum',
  'ESG',
  '키워드',
  '검색',
  '건수',
  '기준',
  '간',
  '전체',
  'ESG',
  '건수',
  '배',
  'ESG',
  '주목',
  '지난해',
  '전',
  '세계',
  '코로나',
  'COVID',
  '전염병',
  '경제',
  '시스템',
  '전반',
  '충격',
  '정부',
  '기업',
  '그간',
  '재무',
  '리스크',
  '요인',
  '관리',
  '필요',
  '절감',
  '주요',
  '기업',
  '금융',
  '사',
  'ESG',
  '전사',
  '리스크',
  '관리',
  '전략',
  '기획',
  '차원',
  '시작',
  '올해',
  '축년',
  '기업',
  '금융',
  '법조',
  '다양',
  '분야',
  '활동',
  'ESG',
  '전문가',
  '초빙',
  'ESG',
  '규제',
  '환경',
  '변화',
  '기업',
  '금융',
  '길',
  '주제',
  '대담',
  '진행',
  '이번',
  '대담',
  '권기준',
  'SK',
  '수',
  '펙',
  '협의회',
  'SV',
  '위원회',
  '부사장',
  '김성우',
  '김',
  '애',
  '장법',
  '사무소',
  '환경',
  '에너지',
  '연구',
  '소장',
  '우상현',
  'KB',
  '은행',
  'CIB',
  '기업',
  '투자',
  '금융',
  '전무',
  'KB',
  '증권',
  'IB',
  '투자',
  '은행',
  '부사장',
  '윤석모',
  '삼성증권',
  '리

In [7]:
import math
import pandas as pd

In [8]:
count_table = pd.DataFrame(columns = ['noun','count','total','probability'])
pmi_table = pd.DataFrame(columns = ['noun1','noun2','co-occurence','pmi'])

In [ ]:
corp = tokenized_data

dic={}
tot=0

dic2d={}
for d in tqdm(corp):
    for w in d:
        tot+=1
        if w not in dic:
            dic[w]=1
        else:
            dic[w]+=1

    l=len(d)
    for i1 in range(0,l-1):
        for i2 in range(i1+1, l):
            w1=d[i1]
            w2=d[i2]
            if w1 not in dic2d:
                dic2d[w1]={}
            if w2 not in dic2d[w1]:
                dic2d[w1][w2]=1
            else:
                dic2d[w1][w2]+=1

            if w2 not in dic2d:
                dic2d[w2]={}
            if w1 not in dic2d[w2]:
                dic2d[w2][w1]=1
            else:
                dic2d[w2][w1]+=1

for key in tqdm(dic.keys()):
    pr = dic[key]/tot
    #print(key, dic[key], tot, pr)
    aa = {'noun1' : key, 
          'count' : dic[key],
          'total' : tot,
          'probability' : pr}
    count_table.append(aa)
    #print('-'*50)


pmi={}
for key1 in tqdm(dic2d.keys()):
    dsub=dic2d[key1]
    pmi[key1]={}
    for key2 in dsub.keys():
        pmiValue=math.log((dic2d[key1][key2]/tot)/((dic[key1]/tot)*(dic[key2]/tot))) #pmi 수식(값이 높을수록 관련성이 높다.)
        #print(key1, key2, dic2d[key1][key2], pmiValue)
        bb = {'noun1' : key1, 
              'noun2' : key2,
              'co-occurence' : dic2d[key1][key2],
              'pmi' : pmiValue}
        pmi_table.append(bb)
        pmi[key1][key2]=pmiValue

  0%|          | 0/3472 [00:00<?, ?it/s]

In [5]:
# 단순 카운트
dic

{'우려': 2, '반발': 3, '부정': 2, '곤란': 2, '기대': 2, '필요': 3, '적극': 2, '시급': 2}

In [7]:
# pmi
pmi

{'우려': {'반발': 1.791759469228055,
  '부정': 1.5040773967762742,
  '곤란': 1.5040773967762742},
 '반발': {'우려': 1.791759469228055,
  '부정': 1.791759469228055,
  '곤란': 1.791759469228055},
 '부정': {'우려': 1.5040773967762742,
  '반발': 1.791759469228055,
  '곤란': 1.5040773967762742},
 '곤란': {'우려': 1.5040773967762742,
  '반발': 1.791759469228055,
  '부정': 1.5040773967762742},
 '기대': {'필요': 1.791759469228055,
  '적극': 1.5040773967762742,
  '시급': 1.5040773967762742},
 '필요': {'기대': 1.791759469228055,
  '적극': 1.791759469228055,
  '시급': 1.791759469228055},
 '적극': {'기대': 1.5040773967762742,
  '필요': 1.791759469228055,
  '시급': 1.5040773967762742},
 '시급': {'필요': 1.791759469228055,
  '적극': 1.5040773967762742,
  '기대': 1.5040773967762742}}